In [17]:
import os
import sys
from nilearn import image
from pathlib import Path
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import bids
from bids import BIDSLayout

dir2 = os.path.abspath('../..')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: 
    sys.path.append(dir1)

from tc2see import load_data

from noise_ceiling import (
    compute_ncsnr,
    compute_nc,
)

In [18]:
dataset_root = Path('E:\\fmri_processing\\results')
tc2see_version = 3
dataset_path = dataset_root
derivatives_path = dataset_path / 'derivatives_TC2See'
data_path = derivatives_path / 'fmriprep'

dataset_layout = BIDSLayout(dataset_path / 'TC2See')
derivatives_layout = BIDSLayout(derivatives_path / 'fmriprep', derivatives=True, validate = False)

c:\Users\James\anaconda3\envs\representationsLab\Lib\site-packages\bids\layout\layout.py:516: UserWarning: Derivative indexing was requested, but no valid datasets were found in the specified locations ([WindowsPath('E:/fmri_processing/results/derivatives_TC2See/fmriprep/derivatives')]). Note that all BIDS-Derivatives datasets must meet all the requirements for BIDS-Raw datasets (a common problem is to fail to include a 'dataset_description.json' file in derivatives datasets).
Example contents of 'dataset_description.json':
{"Name": "Example dataset", "BIDSVersion": "1.0.2", "GeneratedBy": [{"Name": "Example pipeline"}]}
  warnings.warn("Derivative indexing was requested, but no valid "


In [20]:
exclude_list = [13]
subject_nums = [sub for sub in range(5,35) if sub not in exclude_list]
# subject_nums = [34]

for sub in subject_nums:
    subject_no = str(sub) if sub >= 10 else '0'+str(sub)
    num_runs = 6
    tr = 2
    task = "bird"
    space = 'T1w'

    load_data_params = dict(
        path = data_path / f'tc2see-v{tc2see_version}-bold.hdf5', 
        subject = f'sub-{subject_no}',
        tr_offset = 6 / tr,
        run_normalize='linear_trend',
        interpolation=False,
    )

    training_run_ids = list(range(num_runs))
    bold_train, stimulus_ids_train, mask, affine = load_data(
        **load_data_params,
        run_ids = training_run_ids
    )

    ncsnr = compute_ncsnr(bold_train, stimulus_ids_train) # Compute noise ceiling noise ratio
    nc = compute_nc(ncsnr, num_averages=1)
    nc_volume = np.zeros_like(mask, dtype=float)
    nc_volume[mask] = nc

    bird_data_folder = Path(f'E:/fmri_processing/results/derivatives_TC2See/fmriprep/sub-{subject_no}')
    func_file = Path(bird_data_folder / f'func/sub-{subject_no}_task-bird_run-1_space-T1w_desc-preproc_bold.nii.gz')
    vis_file = Path(f'E:/fmri_processing/results/visualization/sub-{subject_no}')

    func_data = image.load_img(func_file) # load functional data which has a temporal dimension
    mean_func_data = image.mean_img(func_data) # take the average to just get a 3-dimensional representation

    nc_volume_img = nib.Nifti1Image(nc_volume, affine=mean_func_data.affine)

    if not Path(vis_file).exists():
        Path(vis_file).mkdir(exist_ok=True)

    print(f"Saving files for subject {sub} at: {vis_file}")
    nib.save(nc_volume_img, vis_file / f'sub-{subject_no}_nc_volume.nii.gz')

Saving files for subject 34 at: E:\fmri_processing\results\visualization\sub-34
